### Step1
### Filtering Educational Tweets out of 500GB data of 100M Tweets

In [1]:
import os
import time
import subprocess

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from pyspark.sql import functions as F
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [ ]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

### Running API/Column Based Filtering

In [6]:
#only english language
df_filter_withConditions = df_full.filter(col('lang') == 'en') 

In [7]:
#no sensitive content
df_filter_withConditions = df_filter_withConditions.filter((col('possibly_sensitive') == False) | col('possibly_sensitive').isNull())

In [8]:
#no truncated tweets
df_filter_withConditions = df_filter_withConditions.filter((col('truncated') == False) | col('truncated').isNull())

In [ ]:
df_filter_withConditions.count()

78307824

##### At this point filtered out based on tweet characteristics and got 78M / 100M

### Using filter words from Chatgpt, google, wikipedia and basic intuition to filter tweets which has these words in tweet_text

In [9]:
filter_words = ['education', 'educational', 'kindergarten', 'primary school', 'local school', 'private school'
               , 'public school', 'high school','college', 'academic institution', 'academics', 'k12', 'k-12',
                'community college', 'junior college', 'senior high school', 'junior high school', 'middle school',
                'elementry school', 'Nursery', 'elementry education', 'masters degree', 'masters', 'bachelors', 
                'undergrad', 'undergraduate', 'graduate', 'post-graduate', 'postgraduate', 'postgrad', 'post-grad',
                'grad', 'schooling', 'studying','teaching', 'learning', 'educational research', 'research', 'classroom',
                'home work', 'home-work', 'preschool', 'educationalist', 'syllabus', 'study load', 'peer pressure', 
                'textbook', 'workload', 'educational stress', 'academic success', 'online education', 'e-learning', 
                'e learning', 'literacy', 'pen', 'books', 'professor', 'research associate', 'post-doctorate', 'PhD',
                'postdoctorate', 'professional education', 'academic training', 'schooling', 'learning', 'studying',
                'p-12', 'p12', 'primary school', 'secondary education', 'secondary school', 'nursery', 'chemistry',
                'biology', 'physics', 'physical education', 'study', 'academic achievements', 'nobel prize',
                'academic scholar', 'merit scholarships', 'scholarships', 'tuition', 'tuition fee', 'study',
                'academic conference', 'educational conference'
               ]

In [10]:
len(filter_words)

87

In [11]:
filter_words = '|'.join(["(" + c +")" for c in filter_words])

In [12]:
filter_words

'(education)|(educational)|(kindergarten)|(primary school)|(local school)|(private school)|(public school)|(high school)|(college)|(academic institution)|(academics)|(k12)|(k-12)|(community college)|(junior college)|(senior high school)|(junior high school)|(middle school)|(elementry school)|(Nursery)|(elementry education)|(masters degree)|(masters)|(bachelors)|(undergrad)|(undergraduate)|(graduate)|(post-graduate)|(postgraduate)|(postgrad)|(post-grad)|(grad)|(schooling)|(studying)|(teaching)|(learning)|(educational research)|(research)|(classroom)|(home work)|(home-work)|(preschool)|(educationalist)|(syllabus)|(study load)|(peer pressure)|(textbook)|(workload)|(educational stress)|(academic success)|(online education)|(e-learning)|(e learning)|(literacy)|(pen)|(books)|(professor)|(research associate)|(post-doctorate)|(PhD)|(postdoctorate)|(professional education)|(academic training)|(schooling)|(learning)|(studying)|(p-12)|(p12)|(primary school)|(secondary education)|(secondary school

In [13]:
df_filter_withConditions = df_filter_withConditions.withColumn('lower_text', lower(col('text')))\
                                                    .withColumn('lower_tweet_text', lower(col('tweet_text')))

In [14]:
df_filtered = df_filter_withConditions.where(col('lower_tweet_text').rlike(filter_words))

##### Got 38M tweets which has educational keywords

In [ ]:
df_filtered.write.mode('overwrite').parquet("gs://msca-bdp-students-bucket/shared_data/safzal1/finalprojectdata_filtered2")

In [54]:
df3 = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/safzal1/finalprojectdata_filtered2')

### Now we will ensure that the tweet as well as when it is retweeted/reposted, education is talked about, by ensuring that education word is used atleast once

In [15]:
df_filtered2 = df_filtered.where(col('lower_text').rlike('education'))

In [ ]:
df_filtered2.count()

1731801

### Here we have 1.73M tweets to run analysis

In [18]:
df_filtered2.write.parquet("gs://msca-bdp-students-bucket/shared_data/safzal1/finalprojectdata_filtered_1.7M_3")